In [1]:
include("../latentplan/datasets/sequence.jl")
include("../latentplan/utils/setup.jl")
include("../latentplan/models/vqvae.jl")

using .Sequence: SequenceDataset, normalize_joined_single, get_item, DataLoader
using .Setup: parser
using ArgParse: ArgParseSettings, @add_arg_table!, parse_args
using .VQVAE
using Statistics: mean
using Knet
using Printf

No module named 'flow'
No module named 'carla'
pybullet build time: Nov 12 2022 16:05:06


In [2]:
losssum(prediction) = mean(prediction[2] + prediction[3] + prediction[4])

function vq_train(config, model::VQContinuousVAE, dataset; n_epochs=1, log_freq=100)
    # set optimizers
    opt_decay = AdamW(lr=config["learning_rate"], beta1=config["betas"][1], beta2=config["betas"][2], weight_decay=config["weight_decay"], gclip=config["grad_norm_clip"])
    opt_no_decay = AdamW(lr=config["learning_rate"], beta1=config["betas"][1], beta2=config["betas"][2], weight_decay=0.0, gclip=config["grad_norm_clip"])

    for p in paramlist_decay(model)
        p.opt = clone(opt_decay)
    end
    for p in paramlist_no_decay(model)
        p.opt = clone(opt_no_decay)
    end

    n_tokens = 0
    loader = DataLoader(dataset; shuffle=true, batch_size=config["batch_size"])

    for epoch in 1:n_epochs
        losses = []
        for (it, batch) in enumerate(loader)
            y = batch[end-1]
            n_tokens += cumprod(size(y))

            if n_tokens < config["warmup_tokens"]
                # linear warmup
                lr_mult = float(n_tokens) / float(max(1, config["warmup_tokens"]))
            else
                # cosine learning rate decay
                progress = float(n_tokens - config["warmup_tokens"]) / float(
                    max(1, config["final_tokens"] - config["warmup_tokens"])
                )
                lr_mult = max(0.1, 0.5 * (1.0 + cos(pi * progress)))
            end

            if config["lr_decay"]
                lr = config["learning_rate"] * lr_mult
                # TODO: param_group learning rate
                for p in paramlist(model)
                    p.opt.lr = lr
                end
            else
                lr = config["learning_rate"]
            end

            # forward the model
            total_loss = @diff losssum(model(batch...))
            push!(losses, value(total_loss))
            for p in paramlist(model)
                update!(p, grad(total_loss, p))
            end
        end
    end
end

vq_train (generic function with 1 method)

In [3]:
s = ArgParseSettings()
@add_arg_table! s begin
    "--dataset"
        help = "which environment to use"
        arg_type = String
        default = "halfcheetah-medium-expert-v2"
    "--exp_name"
        help = "name of the experiment"
        arg_type = String
        default = "debug"
    "--seed"
        help = "seed"
        arg_type = Int
        default = 42
    "--config"
        help = "relative jl file path with configurations"
        arg_type = String
        default = "../config/vqvae.jl"
end

#######################
######## setup ########
#######################

super_args = parse_args([], s)
args = parser(super_args, experiment="train")

[ utils/setup ] Reading config: ../config/vqvae.jl:halfcheetah_medium_expert_v2
/Users/enes/logs/halfcheetah-medium-expert-v2/debug/ already exists. Proceeding...
Made directory/Users/enes/logs/halfcheetah-medium-expert-v2/debug/


OrderedCollections.OrderedDict{String, Any} with 47 entries:
  "model"                      => "VQTransformer"
  "tag"                        => "experiment"
  "state_conditional"          => true
  "N"                          => 100
  "discount"                   => 0.99
  "n_layer"                    => 4
  "n_head"                     => 4
  "n_epochs_ref"               => 50
  "n_saves"                    => 3
  "logbase"                    => "~/logs/"
  "device"                     => "cuda"
  "K"                          => 512
  "latent_step"                => 3
  "n_embd"                     => 128
  "trajectory_embd"            => 512
  "batch_size"                 => 512
  "learning_rate"              => 0.0002
  "lr_decay"                   => false
  "seed"                       => 42
  "embd_pdrop"                 => 0.1
  "resid_pdrop"                => 0.1
  "attn_pdrop"                 => 0.1
  "step"                       => 1
  "subsampled_sequence_length" => 25
  "

# Dataset

In [4]:
env_name = occursin("-v", args["dataset"]) ? args["dataset"] : args["dataset"] * "-v0"

# env params
sequence_length = args["subsampled_sequence_length"] * args["step"]
args["logbase"] = expanduser(args["logbase"])
args["savepath"] = expanduser(args["savepath"])
if !isdir(args["savepath"])
    mkpath(args["savepath"])
end

dataset = SequenceDataset(
    env_name;
    penalty=args["termination_penalty"], 
    sequence_length=sequence_length, 
    step=args["step"], 
    discount=args["discount"], 
    disable_goal=args["disable_goal"], 
    normalize_raw=args["normalize"], 
    normalize_reward=args["normalize_reward"],
    max_path_length=args["max_path_length"],
)

obs_dim = dataset.observation_dim
act_dim = dataset.action_dim
if args["task_type"] == "locomotion"
    transition_dim = obs_dim+act_dim+3
else
    transition_dim = 128+act_dim+3
end

block_size = args["subsampled_sequence_length"] * transition_dim # total number of dimensionalities for a maximum length sequence (T)

print(
    "Dataset size: $(length(dataset)) |
    Joined dim: $transition_dim
    observation: $obs_dim, action: $act_dim | Block size: $block_size"
)

[ datasets/sequence ] Sequence length: 25 | Step: 1 | Max path length: 1000
[ datasets/sequence ] Loading...


/Users/enes/.julia/conda/3/lib/python3.10/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
load datafile: 100%|█████████████████████████████| 9/9 [00:05<00:00,  1.67it/s]
Generating dataset 100%|█████████████████████████████████| Time: 0:00:00


✓
[ datasets/sequence ] Segmenting...
✓
Dataset size: 48 |
    Joined dim: 26
    observation: 17, action: 6 | Block size: 650

# Model

In [5]:
model_config = deepcopy(args)
model_config["block_size"] = block_size
model_config["observation_dim"] = obs_dim
model_config["action_dim"] = act_dim
model_config["transition_dim"] = transition_dim
model_config["n_embd"] = args["n_embd"] * args["n_head"]
model_config["vocab_size"] = args["N"]

model = VQContinuousVAE(model_config)

VQContinuousVAE(VQStepWiseTransformer(512, 512, 17, 624, 512, 26, 650, 17, 6, 26, 3, true, "uniform", Main.VQVAE.Common.Chain((Main.VQVAE.Transformers.Block(Main.VQVAE.Transformers.Common.LayerNorm(P(Vector{Float32}(512)), P(Vector{Float32}(512)), 1.0e-5), Main.VQVAE.Transformers.Common.LayerNorm(P(Vector{Float32}(512)), P(Vector{Float32}(512)), 1.0e-5), Main.VQVAE.Transformers.CausalSelfAttention(Main.VQVAE.Transformers.Common.Linear(P(Matrix{Float32}(512,512)), P(Vector{Float32}(512)), 0), Main.VQVAE.Transformers.Common.Linear(P(Matrix{Float32}(512,512)), P(Vector{Float32}(512)), 0), Main.VQVAE.Transformers.Common.Linear(P(Matrix{Float32}(512,512)), P(Vector{Float32}(512)), 0), Main.VQVAE.Transformers.Common.Linear(P(Matrix{Float32}(512,512)), P(Vector{Float32}(512)), 0), [1.0 1.0 … 1.0 1.0; 0.0 1.0 … 1.0 1.0; … ; 0.0 0.0 … 1.0 1.0; 0.0 0.0 … 0.0 0.0], 0.1, 0.1, 4), Main.VQVAE.Transformers.Common.Chain((Main.VQVAE.Transformers.Common.Linear(P(Matrix{Float32}(2048,512)), P(Vector{Floa

In [6]:
paramlist(model)

LoadError: MethodError: no method matching paramlist(::Main.VQVAE.Common.Chain)
[0mClosest candidates are:
[0m  paramlist([91m::VQEmbedding[39m) at ~/Projects/comp441/latentplan.jl/latentplan/models/vqvae.jl:62
[0m  paramlist([91m::VQEmbeddingMovingAverage[39m) at ~/Projects/comp441/latentplan.jl/latentplan/models/vqvae.jl:90
[0m  paramlist([91m::VQStepWiseTransformer[39m) at ~/Projects/comp441/latentplan.jl/latentplan/models/vqvae.jl:215
[0m  ...